In [2]:
import pickle
from keras.models import load_model
import tensorflow as tf
import cv2
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
from keras.optimizers import Adam
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import os
import json
import requests
from bs4 import BeautifulSoup
from PIL import Image
import re
import smtplib
from email.mime.text import MIMEText
import wget
import zipfile


In [3]:
# Define the source URL for the dataset
source_url = "https://sites.google.com/site/pornographydatabase/"
response = requests.get(source_url)
soup = BeautifulSoup(response.text, "html.parser")
image_urls = []
for img in soup.find_all("img"):
    image_url = img.get("src")
    if image_url:
        image_urls.append(image_url)

print(image_urls)

['//ssl.gstatic.com/images/branding/googlelogo/2x/googlelogo_color_74x24dp.png']


In [ ]:
# Download the dataset
#if download dataset you don't have to download again 
url = "https://figshare.com/ndownloader/files/27771210"
downloaded_file = wget.download(url)

In [38]:
# Define the extraction directory
extract_dir = "/content/extracted_files"
os.makedirs(extract_dir, exist_ok=True)

# Specify the path to the zip file
zip_file_path = "27771210"

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Define the image data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(os.path.join(extract_dir, 'image dataset', 'train'), target_size=(299, 299), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory(os.path.join(extract_dir, 'image dataset', 'test'), target_size=(299, 299), batch_size=32, class_mode='binary')

# Load the pre-trained InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add fully-connected layers
x = Dense(1024, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# Add a final output layer with sigmoid activation for binary classification
predictions = Dense(1, activation='sigmoid')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Define a callback to save the model with the best validation accuracy
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', verbose=1, mode='min', save_best_only=True)

# Train the model
history = model.fit(train_generator, epochs=6, steps_per_epoch=10, validation_data=test_generator, validation_steps=10, callbacks=[checkpoint])

# Save the model's history to a pickle file
with open('history.pickle', 'wb') as f:
    pickle.dump(history.history, f)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, steps=10)
print('Test accuracy:', test_acc)

Found 22306 images belonging to 2 classes.
Found 8881 images belonging to 2 classes.
Epoch 1/6
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.6375 - loss: 0.6257 
Epoch 1: val_loss improved from inf to 0.41901, saving model to best_model.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 587s 24s/step - accuracy: 0.6483 - loss: 0.6169 - val_accuracy: 0.8125 - val_loss: 0.4190
Epoch 2/6
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14s/step - accuracy: 0.8738 - loss: 0.2855 
Epoch 2: val_loss did not improve from 0.41901
10/10 ━━━━━━━━━━━━━━━━━━━━ 181s 19s/step - accuracy: 0.8759 - loss: 0.2826 - val_accuracy: 0.7000 - val_loss: 0.8947
Epoch 3/6
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.8939 - loss: 0.2690 
Epoch 3: val_loss did not improve from 0.41901
10/10 ━━━━━━━━━━━━━━━━━━━━ 149s 15s/step - accuracy: 0.8951 - loss: 0.2662 - val_accuracy: 0.8469 - val_loss: 0.5100
Epoch 4/6
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.9217 - loss: 0.2126 
Epoch 4: val_loss improved from 0.41901 to 0.38444, s

In [8]:
# Load the trained model
model = load_model('best_model.keras')

# Function to preprocess frames
def preprocess_frame(frame):
    # Resize the frame to match the input size of the model
    frame = cv2.resize(frame, (299, 299))
    # Rescale pixel values to the range [0, 1]
    frame = frame / 255.0
    return frame

# Function to classify frames
def classify_frame(frame):
    # Preprocess the frame
    processed_frame = preprocess_frame(frame)
    # Add batch dimension as the model expects batches of images
    processed_frame = np.expand_dims(processed_frame, axis=0)
    # Predict using the model
    prediction = model.predict(processed_frame)
    # Return the prediction
    return prediction

# Function to extract frames from a video
def extract_frames(video_path, output_dir_porn, output_dir_non_porn, threshold=0.8):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    os.makedirs(output_dir_porn, exist_ok=True)
    os.makedirs(output_dir_non_porn, exist_ok=True)
    
    for i in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break
        
        prediction = classify_frame(frame)
        # Print the prediction for debugging purposes
        print(f"Frame {i}: Prediction = {prediction}")
        
        # Check if the prediction indicates pornography
        if prediction[0][0] > threshold:
            # Save the frame to the porn directory
            frame_filename = os.path.join(output_dir_porn, f"frame_{i}.jpg")
        else:
            # Save the frame to the non-porn directory
            frame_filename = os.path.join(output_dir_non_porn, f"frame_{i}.jpg")
        
        cv2.imwrite(frame_filename, frame)
    
    cap.release()

# Example usage
video_path = r"C:\Users\mazen\Downloads\Telegram Desktop\video_2024-06-08_00-09-52.mp4" 
output_dir_porn = 'frames_with_pornography'
output_dir_non_porn = 'frames_without_pornography'
extract_frames(video_path, output_dir_porn, output_dir_non_porn)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Frame 0: Prediction = [[0.23584104]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
Frame 1: Prediction = [[0.14637391]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Frame 2: Prediction = [[0.1303405]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Frame 3: Prediction = [[0.03494709]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Frame 4: Prediction = [[0.05474848]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Frame 5: Prediction = [[0.04569574]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Frame 6: Prediction = [[0.08828069]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
Frame 7: Prediction = [[0.1019318]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
Frame 8: Prediction = [[0.05685724]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
Frame 9: Prediction = [[0.06724582]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Frame 10: Prediction = [[0.11425778]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 11: Prediction = [[0.06606114]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 12: Prediction = [[0.04172124]]
1/1 ━━━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Frame 74: Prediction = [[0.4028402]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 75: Prediction = [[0.22476713]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Frame 76: Prediction = [[0.48211426]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 77: Prediction = [[0.7048791]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
Frame 78: Prediction = [[0.4592095]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 79: Prediction = [[0.79488647]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 80: Prediction = [[0.73627186]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 81: Prediction = [[0.7887838]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 82: Prediction = [[0.36933786]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 83: Prediction = [[0.23122513]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
Frame 84: Prediction = [[0.2633845]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
Frame 85: Prediction = [[0.22687122]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
Frame 86: Prediction = [[0.41884863]]
1/1 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 147: Prediction = [[0.1503663]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
Frame 148: Prediction = [[0.14218485]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Frame 149: Prediction = [[0.12774976]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
Frame 150: Prediction = [[0.14128956]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
Frame 151: Prediction = [[0.16248767]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
Frame 152: Prediction = [[0.17704582]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 153: Prediction = [[0.14890051]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 154: Prediction = [[0.0907922]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
Frame 155: Prediction = [[0.1103613]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
Frame 156: Prediction = [[0.11579919]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 157: Prediction = [[0.12229062]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
Frame 158: Prediction = [[0.1563729]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 159: Prediction = [[0.1

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
Frame 220: Prediction = [[0.03691749]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
Frame 221: Prediction = [[0.07008822]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
Frame 222: Prediction = [[0.20317934]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
Frame 223: Prediction = [[0.24249394]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
Frame 224: Prediction = [[0.13777643]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
Frame 225: Prediction = [[0.37239602]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 226: Prediction = [[0.33421525]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
Frame 227: Prediction = [[0.11632163]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
Frame 228: Prediction = [[0.1461508]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
Frame 229: Prediction = [[0.15690576]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
Frame 230: Prediction = [[0.1343521]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
Frame 231: Prediction = [[0.19762799]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 232: Prediction = [[0

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 293: Prediction = [[0.16306528]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
Frame 294: Prediction = [[0.15606949]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
Frame 295: Prediction = [[0.12973215]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
Frame 296: Prediction = [[0.1362497]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 297: Prediction = [[0.10658991]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
Frame 298: Prediction = [[0.19710769]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
Frame 299: Prediction = [[0.461696]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
Frame 300: Prediction = [[0.49753076]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 301: Prediction = [[0.5485934]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
Frame 302: Prediction = [[0.26498076]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
Frame 303: Prediction = [[0.25905445]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
Frame 304: Prediction = [[0.37588307]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
Frame 305: Prediction = [[0.1

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
Frame 439: Prediction = [[0.03067831]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
Frame 440: Prediction = [[0.02418668]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
Frame 441: Prediction = [[0.02552997]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
Frame 442: Prediction = [[0.02839315]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
Frame 443: Prediction = [[0.02535778]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
Frame 444: Prediction = [[0.02695459]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
Frame 445: Prediction = [[0.0253018]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
Frame 446: Prediction = [[0.0235433]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
Frame 447: Prediction = [[0.02768358]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
Frame 448: Prediction = [[0.0189584]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step
Frame 449: Prediction = [[0.05535328]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
Frame 450: Prediction = [[0.1410583]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
Frame 451: Prediction = [[0.2

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
Frame 512: Prediction = [[0.03373205]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
Frame 513: Prediction = [[0.02132115]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
Frame 514: Prediction = [[0.0198016]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
Frame 515: Prediction = [[0.01486914]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
Frame 516: Prediction = [[0.05781807]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
Frame 517: Prediction = [[0.08846464]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step
Frame 518: Prediction = [[0.08547128]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
Frame 519: Prediction = [[0.10008176]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
Frame 520: Prediction = [[0.03579949]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
Frame 521: Prediction = [[0.03310543]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
Frame 522: Prediction = [[0.0330102]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
Frame 523: Prediction = [[0.04678811]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
Frame 524: Prediction = [[0

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
Frame 585: Prediction = [[0.06539974]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
Frame 586: Prediction = [[0.02625994]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
Frame 587: Prediction = [[0.03145625]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
Frame 588: Prediction = [[0.02810788]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
Frame 589: Prediction = [[0.02978019]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
Frame 590: Prediction = [[0.02914524]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
Frame 591: Prediction = [[0.02813518]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step
Frame 592: Prediction = [[0.02204487]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
Frame 593: Prediction = [[0.02642185]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
Frame 594: Prediction = [[0.02734432]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
Frame 595: Prediction = [[0.03496929]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step
Frame 596: Prediction = [[0.04351447]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
Frame 597: Prediction = [

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Frame 658: Prediction = [[0.05472203]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 659: Prediction = [[0.09296558]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Frame 660: Prediction = [[0.05276325]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 661: Prediction = [[0.03672423]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 662: Prediction = [[0.07021882]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 663: Prediction = [[0.20368968]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 664: Prediction = [[0.1418865]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
Frame 665: Prediction = [[0.19798116]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 666: Prediction = [[0.09459252]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 667: Prediction = [[0.30497634]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 668: Prediction = [[0.11457734]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 669: Prediction = [[0.10682601]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 670: Prediction = [[

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 731: Prediction = [[0.01481491]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Frame 732: Prediction = [[0.01453875]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 733: Prediction = [[0.01553648]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 734: Prediction = [[0.01558462]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
Frame 735: Prediction = [[0.01610494]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Frame 736: Prediction = [[0.01612629]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 737: Prediction = [[0.01605948]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Frame 738: Prediction = [[0.0153079]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 739: Prediction = [[0.01461364]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 740: Prediction = [[0.01439444]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Frame 741: Prediction = [[0.01432748]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Frame 742: Prediction = [[0.04682922]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
Frame 743: Prediction = [[